In [1]:
# imports and definitions

import time 
import threading
import serial

def btoa(bstr): #returns an ascii string from a bytes object
    btoadat = bstr.decode('ascii')
    return btoadat
def atob(strb): #returns a bytes object from an ascii string
    atobdat = strb.encode('ascii')
    return atobdat

def pumpread(ser, pumpid, cmd): # interacts with the pump
    t_ini=(time.time())
    ser.open()
    ser.write(atob(str(pumpid) + str(cmd) + '\x0D'))
    pumpdat = btoa(ser.read_until(b'\x03',100))
    ser.close()
    return pumpdat[1:-1]

def getweight(ser): # interacts with the Ohaus balance
    ser.open()
    ser.write(b'P\r\n')
    weight=float(btoa(ser2.read_until(b'\r\n',100)).split()[0]) #print time, too
    ser.close()
    return weight

# name and prepare the syringe pump

serP = serial.Serial('/dev/cu.usbserial-10') #for SyringeOne pump StarTech attach
serP.baudrate = 19200; serP.timeout=0.1
serP.close()

print(pumpread(serP, 2, '*RESET')) #clears previous programs, which is not really necessary
serP.close() 

SerialException: [Errno 2] could not open port /dev/cu.usbserial-10: [Errno 2] No such file or directory: '/dev/cu.usbserial-10'

In [ ]:
# control pumping by weight readings

# set initials
wini=getweight(ser2) 
tini=time.time()
tstp=time.time()
ii=True

# pump directions
drc=['inf','wdr'] 
print('Direction:', drc[ii], '; Time:', time.time()-tini,'s')

# vol in mL
pumpread(serP,2,'rat 4.0') # pump rate
pumpread(serP,2,'dir %s'%drc[ii]) # initial direction
pumpread(serP,2,'run') # starts pump

# loop pumping over a given time
while (time.time()-tini)<=60: # set run time
    if (time.time()-tstp)>=5: # checks weight every 5 sec
        if abs(wini-getweight(ser2))>=25: # switches direction after meeting 25g change
            pumpread(serP,2,'stp') # pauses pump
            ii=not ii
            print('Direction:', drc[ii], '; Time:', time.time()-tini,'s')
            wini=getweight(ser2) # sets new initial weight once 25g has been met
            pumpread(serP,2,'dir %s'%drc[ii]) # switches directions
            pumpread(serP,2,'run') # restarts pump
        tstp=time.time()
    
pumpread(serP,2,'stp') # pauses pump
print('Finished at', time.time()-tini,'s')

In [ ]:
# stop and close out

serP.close()
print(pumpread(serP, 2, '*RESET')) 
pumpread(serP,2,'stp')